In [69]:
import random
import bisect
import tracemalloc

def Generate_Population(size,N):
	population = []
	for individual in range(size):
		individual = []
		for i in range(N):
			individual.append(random.randint(0,N-1))
		population.append(individual)	
	return population

def f(current_state):   
    """
    funzione obiettivo che conta il numero di regine correttamente posizionate,
    ovvero che non attaccano nessuna regina sottostante
    """
    correctly_positioned = 0
    for row in range(N-1):
        no_attack_on_lower_rows = 0
        for lower_rows in range(row+1,N):
            if current_state[row] != current_state[lower_rows] and abs(current_state[lower_rows]-current_state[row]) != abs(lower_rows-row):
                no_attack_on_lower_rows += 1
        if no_attack_on_lower_rows == N-1-row:
            correctly_positioned += 1
    if correctly_positioned == N-1:
        correctly_positioned += 1
    return correctly_positioned

def Prob_List(population):
    """
    ritorna una lista dove gli indici sono gli indici per trovare gli individui nella popolazione
    e i valori la probabilità cumulata di ogni individuo per la roulette wheel selection
    """
    f_values = []
    for individual in population:
        f_values.append(f(individual))
    fk = sum(f_values)
    fitness = []
    current_f = 0 
    for individual in population:
            current_f = current_f + f(individual)
            fitness.append(current_f / fk)
    
    return fitness, f_values

def Selection(population,fitness,N,probability):
    """
    inserisce nella lista di probabilità cumulata (fitness) un numero random,
    di cui poi estrae l'indice per trovare il corrispondente individuo nella popolazione.
    I posti rimanenti dopo la selezione vengono riempiti poi in modo casuale estraendo da tutta
    la popolazione iniziale
    """
    survived = []
    for i in range(int(len(population)*probability)):     #il range mi dà il numero di individui da selezionare
        probabilities = fitness.copy()
        r = random.random()
        bisect.insort(probabilities,r)
        winner_index = probabilities.index(r)
        survived.append(population[winner_index].copy())
    rdm = random.sample([x for x in range (len(population))],k=len(population)-int(len(population)*probability))
    for index in rdm:
        survived.append(population[index].copy())
    return survived

def Crossover(population,probability,N):
    """
    con una certa probabilità, estrae due individui dalla popolazione 
    che taglia in due nello stesso punto e ricombina per ottenere due nuovi individui.
    Gli individui che non si sono accoppiati vengono aggiunti alla fine del ciclo
    """
    sons = []
    for i in range(int(len(population)/2)):
        if random.random() < probability:
            crossover_point = random.randint(1,N-1)   #è possibile che non si verifichi crossover (se lo si vuole evitare porre (1,N-1))
            male = population.pop(random.randint(0,len(population)-1))
            female = population.pop(random.randint(0,len(population)-1))
            son1 = male[0:crossover_point] + female[crossover_point:N]
            son2 = female[0:crossover_point] + male[crossover_point:N]
            sons.append(son1)
            sons.append(son2)
    sons += population.copy()	
    return sons

def Mutation(population,probability,N):
    """
    per ogni individuo con una certa probabilità 
    si modifica la colonna di una regina
    """
    for i in range(len(population)):
        if random.random() < probability:
            row = random.randint(0,N-1)
            possible_col = [x for x in range(N)]
            possible_col.remove(population[i][row])
            new_col = random.choice(possible_col)
            population[i][row] = new_col
    return population




if __name__ == "__main__":
    N = int(input("Dimensione Scacchiera"))
    population_size = int(input("Numero Pari Di Individui"))

    l = []
    tracemalloc.start() 
    for j in range(10):
        new_gen = Generate_Population(population_size,N)
        for i in range(1000):
            fit=Prob_List(new_gen)
            if N in fit[1]:                            #se c'è una soluzione, fermo il ciclo e salvo l'individuo
                winner = new_gen[fit[1].index(N)]      #se voglio eseguire sono una volta uso la parte con gli asterischi
                l.append((i,winner))                   #e metto 1 nel range del ciclo esterno
                #print(i)
                #print(winner, "YOU GOT A WINNER")
                break
            new_gen = Selection(new_gen,fit[0],N,1)
            new_gen = Crossover(new_gen,0.5,N)
            new_gen = Mutation(new_gen,0.4,N)
            
    a = tracemalloc.get_traced_memory()
    tracemalloc.stop()        
    print(len(l),l)
    print(a)

0 []
(12110, 18874)
